Example of multi-tenant user access and management on Waii

Prequisite first: create two postgres databases, which will be used by the example later

Run `sudo -u postgres psql` to open psql

```
-- Connect to PostgreSQL as the postgres superuser

-- Create the first user
CREATE USER waii_test_user1 WITH PASSWORD 'password1';

-- Create the first database owned by waii_test_user1
CREATE DATABASE waii_test_db1 OWNER waii_test_user1;

-- Create the second user
CREATE USER waii_test_user2 WITH PASSWORD 'password2';

-- Create the second database owned by user2
CREATE DATABASE waii_test_db2 OWNER waii_test_user2;

-- Grant privileges (optional but recommended)
GRANT ALL PRIVILEGES ON DATABASE waii_test_db1 TO waii_test_user1;
GRANT ALL PRIVILEGES ON DATABASE waii_test_db2 TO waii_test_user2;
```

You can use other databases (such as snowflake, all the examples below is exchangeabe)

In [23]:
!pip uninstall -y waii-sdk-py
!pip install --upgrade waii-sdk-py pydantic==2.10.3

Found existing installation: waii-sdk-py 1.29.2
Uninstalling waii-sdk-py-1.29.2:
  Successfully uninstalled waii-sdk-py-1.29.2
  Using cached waii_sdk_py-1.29.2-py3-none-any.whl.metadata (2.5 kB)
Using cached waii_sdk_py-1.29.2-py3-none-any.whl (50 kB)

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [4]:
# Now we are going to add some users
# First, let's import all libraries
# Import the main Waii SDK client
from waii_sdk_py import Waii

# Import specific modules
from waii_sdk_py.chat import *
from waii_sdk_py.query import *
from waii_sdk_py.database import *
from waii_sdk_py.semantic_context import *
from waii_sdk_py.chart import *
from waii_sdk_py.history import *
from waii_sdk_py.user import *

# Create admin client
WAII_API_KEY = 'abc123'
WAII_URL = 'http://localhost:9859/api/'

admin_client = Waii()
admin_client.initialize(api_key=WAII_API_KEY, url=WAII_URL)

Important thing to check before you start: you must enable api-key-auth when you start Waii service.

1) If you are using Waii SaaS deployment: you can skip this check.
2) If you are using Docker or K8s deployment, you need to enable it (`api_key_auth_enabled`)
   Check if it is enabled: 

   Open Waii UI using icognito window, you should see it pop up and ask you for an API key to login.

   [ ] check

In [ ]:
# create an organization, let's call it fruit org
admin_client.user.create_org(
    CreateOrganizationRequest(
        organization=Organization(
            id='fruit-org',
            name='Fruit Org'
        )
    )
)

# then create a banana tenant
admin_client.user.create_tenant(
    CreateTenantRequest(
        tenant=Tenant(
            id='banana-tenant',
            name='Banana Tenant',
            org_id='fruit-org'
        )
    )
)

# then create a apple tenant
admin_client.user.create_tenant(
    CreateTenantRequest(
        tenant=Tenant(
            id='apple-tenant',
            name='Apple Tenant',
            org_id='fruit-org'
        )
    )
)

# then create a user for banana tenant
admin_client.user.create_user(
    CreateUserRequest(
        user=User(
            id='banana-user',
            name='Banana User',
            tenant_id='banana-tenant',
            org_id='fruit-org'
        )
    )
)

# then create a user for apple tenant
admin_client.user.create_user(
    CreateUserRequest(
        user=User(
            id='apple-user',
            name='Apple User',
            tenant_id='apple-tenant',
            org_id='fruit-org'
        )
    )
)

# now we have 3 users:
# - admin user
# - banana user
# - apple user

CommonResponse()

In [5]:
# try to list these users
current_users = admin_client.user.list_users(ListUsersRequest(lookup_org_id='fruit-org')).users
for _u in current_users:
    print(_u)
    
# you should see two users:
# - banana user
# - apple user

id='banana-user' name='Banana User' tenant_id='banana-tenant' org_id='fruit-org' variables=None roles=['waii-api-user']
id='apple-user' name='Apple User' tenant_id='apple-tenant' org_id='fruit-org' variables=None roles=['waii-api-user']


In [13]:
# update the users
# add waii-api-user role to banana user and apple user (keep other fields same)
admin_client.user.update_user(
    UpdateUserRequest(
        user=User(
            id='banana-user',
            tenant_id='banana-tenant',
            org_id='fruit-org',
            name='Banana User',
            roles=['waii-api-user']
        )
    )
)
admin_client.user.update_user(
    UpdateUserRequest(
        user=User(
            id='apple-user',
            name='Apple User',
            roles=['waii-api-user'],
            tenant_id='apple-tenant',
            org_id='fruit-org'
        )
    )
)
        
# get users again, you should be able to see the roles updated
# you should be able to see roles=['waii-api-user'] from the response
current_users = admin_client.user.list_users(ListUsersRequest(lookup_org_id='fruit-org')).users
for _u in current_users:
    print(_u)

id='banana-user' name='Banana User' tenant_id='banana-tenant' org_id='fruit-org' variables=None roles=['waii-api-user']
id='apple-user' name='Apple User' tenant_id='apple-tenant' org_id='fruit-org' variables=None roles=['waii-api-user']


In [7]:
# now we try to create api keys for banana user and apple user
# create api key for banana user
# here it uses 'impersonate_user' to create api key for banana user
# note that only super admin / org admin can do impersonate
# the impersonate context will be cleared after the with block
with admin_client.impersonate_user('banana-user'):
    try:
        banana_api_key = admin_client.user.create_access_key(
            CreateAccessKeyRequest(
                # the name is allow you to create named-api-key, you can create multiple api keys for the same user
                # currently all the api keys under the same user are treated as the same user, but in the future we will
                # support different api key permission for same user
                name='default'
            )
        )[0].access_key
    except:
        banana_api_key = admin_client.user.list_access_keys(GetAccessKeyRequest()).access_keys[0].access_key

# create api key for apple user
with admin_client.impersonate_user('apple-user'):
    try:
        apple_api_key = admin_client.user.create_access_key(
            CreateAccessKeyRequest(
                name='default'
            )
        )[0].access_key
    except:
        apple_api_key = admin_client.user.list_access_keys(GetAccessKeyRequest()).access_keys[0].access_key

<Response [400]>
<Response [400]>


In [8]:
# now we have 2 api keys:
# - banana-api-key
# - apple-api-key

# we can create multiple waii clients with different api keys
# and use them to access different tenants

# create a waii client for banana user
banana_client = Waii()
banana_client.initialize(api_key=banana_api_key, url=WAII_URL)

# create a waii client for apple user
apple_client = Waii()
apple_client.initialize(api_key=apple_api_key, url=WAII_URL)

# now try to get user info from banana client
banana_user_info = banana_client.user.get_user_info(GetUserInfoRequest())
print(banana_user_info)

# try to get user info from apple client
apple_user_info = apple_client.user.get_user_info(GetUserInfoRequest())
print(apple_user_info)

# you should see the user info from banana client returns the banana user info, 
# and the apple client returns the apple user info

# Note here: the waii client is very lightweight, it does not hold any state (just a few string variables to store the API key and url)
# so you can create as many as you want, and use them interchangeably.
# It also doesn't have any long-running session, so you don't need to care about the session expiration // resource consumption.

id='banana-user' name='Banana User' email='banana-user' roles=['waii-api-user'] permissions=['write:databases', 'read:similarity-search-index', 'read:liked-queries', 'write:semantic-context', 'read:databases', 'write:liked-queries', 'usage:api', 'read:semantic-context', 'write:access_key']
id='apple-user' name='Apple User' email='apple-user' roles=['waii-api-user'] permissions=['write:databases', 'read:similarity-search-index', 'read:liked-queries', 'write:semantic-context', 'read:databases', 'write:liked-queries', 'usage:api', 'read:semantic-context', 'write:access_key']


In [12]:
# Now let's create some databases for banana user and apple user
db_conn_1 = DBConnection(
    db_type='postgresql',
    host='localhost',
    port=5432,
    username='waii_test_user1',
    password='password1',
    database='waii_test_db1'
)

db_conn_2 = DBConnection(
    db_type='postgresql',
    host='localhost',
    port=5432,
    username='waii_test_user2',
    password='password2',
    database='waii_test_db2'
)

# add it to banana client
all_conns = banana_client.database.modify_connections(
    ModifyDBConnectionRequest(
        updated=[db_conn_1]
    )
)
print(all_conns)

# you will see something like 
# connectors=[DBConnection(key='postgresql://waii_test_user1@localhost:5432/waii_test_db1'
# now you can save the db_connection_key so we can use it later (such as activate, deletion, etc.)
db1_conn_key = 'postgresql://waii_test_user1@localhost:5432/waii_test_db1'

# now let's add db_conn_2 to apple client
all_conns = apple_client.database.modify_connections(
    ModifyDBConnectionRequest(
        updated=[db_conn_2]
    )
)

db2_conn_key = 'postgresql://waii_test_user2@localhost:5432/waii_test_db2'

connectors=[DBConnection(key='postgresql://waii_test_user1@localhost:5432/waii_test_db1', db_type='postgresql', description=None, account_name=None, username='waii_test_user1', password=None, database='waii_test_db1', warehouse=None, role=None, path=None, host='localhost', port=5432, parameters=None, sample_col_values=True, push=False, db_content_filters=None, embedding_model='text-embedding-ada-002', always_include_tables=None, alias=None, db_access_policy=DBAccessPolicy(read_only=False, allow_access_beyond_db_content_filter=True, allow_access_beyond_search_context=True), host_alias=None, user_alias=None, db_alias=None, client_email=None, content_filters=None, sample_filters=None, secure=True, waii_user_id='banana-user', external_authentication_uri=None, need_external_authentication=None)] diagnostics=None default_db_connection_key=None connector_status={'postgresql://waii_test_user1@localhost:5432/waii_test_db1': DBConnectionIndexingStatus(status='not-started', schema_status={}, inde

In [ ]:
# now you can activate the db_conn_key so the banana client // apple client can use it
banana_client.database.activate_connection(db1_conn_key)
apple_client.database.activate_connection(db2_conn_key)

# if you run a query `SELECT current_database();`, you will see it differently
print("Running query on banana client")
print(banana_client.query.run(RunQueryRequest(query='SELECT current_database();')).rows)
print("Running query on apple client")
print(apple_client.query.run(RunQueryRequest(query='SELECT current_database();')).rows)


Running query on banana client
[{'CURRENT_DATABASE': 'waii_test_db1'}]
Running query on apple client
[{'CURRENT_DATABASE': 'waii_test_db2'}]


In [15]:
# now we demonstrate adding multiple databases to the same user
# let's add database2 to banana client too
all_conns = banana_client.database.modify_connections(
    ModifyDBConnectionRequest(
        updated=[db_conn_2]
    )
)
print([conn.key for conn in all_conns.connectors])

# you will see:
# ['postgresql://waii_test_user1@localhost:5432/waii_test_db1', 
#  'postgresql://waii_test_user2@localhost:5432/waii_test_db2']

# we can activate the db_conn_2 for banana client
banana_client.database.activate_connection(db2_conn_key)

# now you can see the banana client can use db_conn_2
print(banana_client.query.run(RunQueryRequest(query='SELECT current_database();')).rows)

['postgresql://waii_test_user1@localhost:5432/waii_test_db1', 'postgresql://waii_test_user2@localhost:5432/waii_test_db2']
[{'CURRENT_DATABASE': 'waii_test_db2'}]


In [ ]:
# let's delete the db_conn_2 from banana client
all_conns = banana_client.database.modify_connections(
    ModifyDBConnectionRequest(
        removed=[db2_conn_key]
    )
)
# it has db_conn_1 only
print("Deleting db_conn_2 from banana client")
print([conn.key for conn in all_conns.connectors])

# delete the db_conn_1 too
all_conns = banana_client.database.modify_connections(
    ModifyDBConnectionRequest(
        removed=[db1_conn_key]
    )
)
# now the banana client has no database
print("Deleting db_conn_1 from banana client too")
print([conn.key for conn in all_conns.connectors])


Deleting db_conn_2 from banana client
['postgresql://waii_test_user1@localhost:5432/waii_test_db1']
Deleting db_conn_1 from banana client too
[]


In [30]:
print(db_conn_2)

key=None db_type='postgresql' description=None account_name=None username='waii_test_user2' password='password2' database='waii_test_db2' warehouse=None role=None path=None host='localhost' port=5432 parameters=None sample_col_values=True push=False db_content_filters=None embedding_model=None always_include_tables=None alias=None db_access_policy=DBAccessPolicy(read_only=False, allow_access_beyond_db_content_filter=True, allow_access_beyond_search_context=True) host_alias=None user_alias=None db_alias=None client_email=None content_filters=None sample_filters=None


In [34]:
# a helper function to get all tables from a database
def get_all_tables(client: Waii):
    catalog = client.database.get_catalogs(GetCatalogRequest())
    table_names = []
    for db in catalog.catalogs:
        for schema in db.schemas:
            for table in schema.tables:
                table_names.append(table.name)
    return table_names

from waii_sdk_py.database.database import SearchContext

# demonstrate modify the database connection for a user
# let's add db_conn_2 to banana client again
all_conns = banana_client.database.modify_connections(
    ModifyDBConnectionRequest(
        updated=[db_conn_2]
    )
)
time.sleep(30)

# activate the db_conn_2 for banana client
banana_client.database.activate_connection(db2_conn_key)
apple_client.database.activate_connection(db2_conn_key)

# get all tables from banana client
print("# Tables from banana client:")
print(len(get_all_tables(banana_client)))

print("# Tables from apple client:")
print(len(get_all_tables(apple_client)))

# and we will modify the db_conn_2
db_conn_2_limited_tables = db_conn_2.copy(deep=True)
# limit the content to information_schema.tables and information_schema.columns
db_conn_2_limited_tables.content_filters = [
    SearchContext(
        db_name = '*',
        schema_name = 'information_schema',
        table_name = 'tables'
    ),
    SearchContext(
        db_name = '*',
        schema_name = 'information_schema',
        table_name = 'columns'
    )
]

# modify the db_conn_2 with limited tables (only have 2 tables)
banana_client.database.modify_connections(
    ModifyDBConnectionRequest(
        updated=[db_conn_2_limited_tables]
    )
)

# once this is modified, it will take a bit time to take effect
# so let's monitor the status
# it will take a while to take effect (to start indexing), so we will wait for 30 seconds before checking the status
time.sleep(30)

# monitoring status of the modification, and break once the the index is finished
total_wait = 0
while True:
    connector_status = banana_client.database.get_connections().connector_status
    status = connector_status[db2_conn_key]
    if status.status == 'completed':
        break
    time.sleep(1)
    total_wait += 1
    print(f"Waiting for the modification to take effect, total wait time: {total_wait} seconds...")

print("# Tables from banana client after modification:")
print(len(get_all_tables(banana_client)))

print("# Tables from apple client after modification:")
print(len(get_all_tables(apple_client)))

# As you can see, the update of the db_conn_2 is completed, and both of the banana client and apple client will have the same result
# this is because the db_conn_2 is shared by both of the banana client and apple client (still have the same key)
# This leaves a question, if we want to limit the content of the db_conn_2 to banana client only, how to do that?
# let's see the next cell

# Tables from banana client:
9
# Tables from apple client:
9
completed
# Tables from banana client after modification:
2
# Tables from apple client after modification:
2


In [ ]:
# how to give a different view of the same database to different users?
# the answer is to create a (db_alias) for the same database
# let's call it db_conn_2_alias_1 and db_conn_2_alias_2

# alias 1 will only have information_schema.tables (1 table)
db_conn_2_alias_1 = db_conn_2.copy(deep=True)
db_conn_2_alias_1.db_alias = 'db_conn_2_alias_1'
db_conn_2_alias_1.content_filters = [
    SearchContext(
        db_name = '*',
        schema_name = 'information_schema',
        table_name = 'tables'
    )
]

# alias 2 will have information_schema.tables and information_schema.columns (2 tables)
db_conn_2_alias_2 = db_conn_2.copy(deep=True)
db_conn_2_alias_2.db_alias = 'db_conn_2_alias_2'
db_conn_2_alias_2.content_filters = [
    SearchContext(
        db_name = '*',
        schema_name = 'information_schema',
        table_name = 'columns'
    ),
    SearchContext(
        db_name = '*',
        schema_name = 'information_schema',
        table_name = 'tables'
    )
]

# now we add the db_conn_2_alias_1 to banana client, and db_conn_2_alias_2 to apple client
all_conns = banana_client.database.modify_connections(
    ModifyDBConnectionRequest(
        updated=[db_conn_2_alias_1]
    )
)
print("Banana client has the following connections:")
print([conn.key for conn in all_conns.connectors])

all_conns = apple_client.database.modify_connections(
    ModifyDBConnectionRequest(
        updated=[db_conn_2_alias_2]
    )
)

print("Apple client has the following connections:")
print([conn.key for conn in all_conns.connectors])

# again, wait for 60 seconds for the system to index, you should still use the index status api, but 
# this time we just sleep to make it simpler
time.sleep(60)


Banana client has the following connections:
['postgresql://waii_test_user2@localhost:5432/waii_test_db2', 'waii://waii_test_user2@localhost/db_conn_2_alias_1']
Apple client has the following connections:
['postgresql://waii_test_user2@localhost:5432/waii_test_db2', 'waii://waii@host/db_conn_2_alias_2', 'waii://waii_test_user2@localhost/db_conn_2_alias_2']


KeyboardInterrupt: 

In [38]:
# Ha! As you can see, we have another db_connection with a different key
# waii://waii_test_user2@localhost/db_conn_2_alias_1 and 'waii://waii_test_user2@localhost/db_conn_2_alias_2
# which creates a different view of the same database

db_conn_2_alias_1_key = 'waii://waii_test_user2@localhost/db_conn_2_alias_1'
db_conn_2_alias_2_key = 'waii://waii_test_user2@localhost/db_conn_2_alias_2'

# let's try to get number of tables from banana client
print("# Tables from banana client - alias 1:")
banana_client.database.activate_connection(db_conn_2_alias_1_key)
print(len(get_all_tables(banana_client)))

print("# Tables from banana client - alias 2:")
apple_client.database.activate_connection(db_conn_2_alias_2_key)
print(len(get_all_tables(apple_client)))

# Tables from banana client - alias 1:
1
# Tables from banana client - alias 2:
2
